## Metodi per integrare numericamente il semicerchio, cioè per mostrare che:

$\int^{+1}_{-1} \sqrt{1-x^2}\, dx = \pi/2$

In [ ]:
import matplotlib.pyplot as plt
import math
import numpy as np

In [ ]:
def circle(x) :
    return np.sqrt(1-x**2)

In [ ]:
circle(0.5)

In [ ]:
x = np.arange(-1,1.,0.0005)
plt.figure(figsize=[9.6,4.8])   # width and height in inches
plt.plot(x,circle(x),'r')
plt.show()

Ripassiamo il concetto di array NumPy e di operazioni elemento per elemento sull'array tramite **funzioni universali** 
* [array NumPy](07_09.ipynb)
  
e notiamo il **sovraccarico (overloading)** della funzione `circle`. Essa accetta un argomento `float` o `np.array` a seconda del tipo che viene passato.

## 1) Metodo dei trapezi (trapezoid method)

In [ ]:
def trapez(lowlim, higlim, ncalls) :
    if (lowlim > higlim):
        raise Exception('function low limit is larger than high limit')
    else: 
       height = (higlim-lowlim)/ncalls 
       return sum([0.5*height*(circle(i*height + lowlim) + circle((i+1)*height+lowlim)) for i in range(ncalls)]) 

L'ultima riga (autodefinizione di una lista di cui viene fatta una somma) è una **funzione generatrice** ed è un esempio di **programmazione lazy**, cioè "pigra", avara di memoria. In questo ogni elemento della lista viene usato per la somma e immediatamente raccolto nei rifiuti perché non c'è alcuna variabile a cui esso è assegnata. Questo ottimizza la memoria e velocizza l'esecuzione.

In [ ]:
test1 = trapez(1,-1,250)     # wrong on purpose

In [ ]:
%timeit test2 = trapez(-1.,1.,1_000)

Abbiamo introdotto qui il **comando magico %timeit**
* [I comandi magici di Python](07_06.ipynb)

In [ ]:
test2 = trapez(-1.,1.,1_000)
true_value = 0.5*np.pi
print (f'{test2:.30f} estimate of {true_value:.30f}')

## 2) Metodo del campionamento (hit-and-miss method)

In [ ]:
import random

## we need to know the function maximum (1 in this case)
def hit_and_miss(lowlim, higlim, ncalls, fmax=1.) :
    if (lowlim > higlim):
        raise Exception('function low limit is larger than high limit')
    hit = 0
    for i in range(ncalls) :
        r1, r2 = random.uniform(lowlim,higlim), random.uniform(0.,fmax)
        if (r2 < circle(r1)) :
            hit += 1 
    return (hit / ncalls) * ( higlim - lowlim ) * fmax

Gestione delle **eccezioni**, cioè dei possibili errori in runtime, in Python:
* [Tentativi](09_09.ipynb)
* [Eccezioni](09_10.ipynb)

In [ ]:
%timeit test2 = hit_and_miss(-1.,1.,1_000)

Ce lo aspettavamo perché sia il metodo dei trapezi che il metodo hit-and-miss hanno tempi di esecuzione **O(ncalls)**

In [ ]:
test2 = hit_and_miss(-1.,1.,1_000)
print (f'{test2:.30f} estimate of {true_value:.30f}')

In [ ]:
calls, delta_trapez, delta_hit_miss = [], [], []

icall = 10
while (icall < 1_000_000) :
    calls.append(math.log10(icall))
    delta_trapez.append(trapez(-1.,1.,icall) - true_value)
    delta_hit_miss.append(hit_and_miss(-1.,1.,icall) - true_value)
    icall *= 3

In [ ]:
plt.plot(calls,delta_trapez,'bo',linestyle='-.')
plt.plot(calls,delta_hit_miss,'ro',linestyle='-.')

In [ ]:
plt.show()

## Passiamo allo stesso problema in 3 dimensioni (volume dell'emisfero), mostrando che:

$\int\int_{x^2+y^2 < 1} \sqrt{1-x^2-y^2}\, dx \,dy = 2\pi/3$

In [ ]:
def sphere(x,y) :
    return np.sqrt(1-x**2-y**2)

In [ ]:
def trapez_3d(lowlim, higlim, ncalls) :
    if (lowlim > higlim):
        raise Exception('function low limit is larger than high limit')
    nsides = int(np.sqrt(ncalls)) 
    height, integral = (higlim-lowlim)/nsides, 0. 
    for i in range(nsides) :
       for j in range(nsides) :
           if ((i*height + lowlim)**2 + (j*height + lowlim)**2 < 1.) and (((i+1)*height + lowlim)**2 + ((j+1)*height + lowlim)**2 < 1.) : 
              integral += 0.5*height**2*(sphere(i*height + lowlim,j*height + lowlim) + sphere((i+1)*height + lowlim,(j+1)*height + lowlim))
    return integral

**NOTA BENE**:
* `ncalls` equivale ora al numero di quadratini in cui si suddivide il cerchio di base e quindi il numero di lati è $\sqrt{ncalls}$. Se si definissero invece `ncalls` lati, il tempo di processamento diventerebbe **O(ncalls$^2$)**, molto più lungo! 
* In questo caso non è possibile usare un'autodefinizione perché va controllato prima che i punti entro i quali si calcola il volume del prisma siano entro il dominio (altrimenti la radice quadrata dà errore in runtime).

In [ ]:
## we need to know the function maximum (1 in this case)
def hit_and_miss_3d(lowlim, higlim, ncalls, fmax=1.) :
    if (lowlim > higlim):
        raise Exception('function low limit is larger than high limit')
    hit = 0
    for i in range(ncalls) :
        r1x, r1y, r2 = random.uniform(lowlim,higlim), random.uniform(lowlim,higlim), random.uniform(0.,fmax)
        if ((r1x**2+r1y**2) < 1. and r2 < sphere(r1x,r1y)) :
            hit += 1 
    return (hit / ncalls) * ( higlim - lowlim )**2 * fmax

In [ ]:
test3 = hit_and_miss_3d(-1.,1,1_000_000)

In [ ]:
true_value_3d = 2.*np.pi/3.
print (f'{test3:.30f} estimate of {true_value_3d:.30f}')

In [ ]:
calls, delta_trapez, delta_hit_miss = [], [], []

icall = 10
while (icall < 1_000_000) :
    calls.append(math.log10(icall))
    delta_trapez.append(trapez_3d(-1.,1.,icall) - true_value_3d)
    delta_hit_miss.append(hit_and_miss_3d(-1.,1.,icall) - true_value_3d)
    icall *= 3

In [ ]:
plt.plot(calls,delta_trapez,'bo',linestyle='-.')
plt.plot(calls,delta_hit_miss,'ro',linestyle='-.')

In [ ]:
plt.show()

## E in $n$ dimensioni?
$\int_{\Sigma_i\,x_i^2 < 1} \sqrt{1-\Sigma_i\,x_i^2}\, d^n x_i =$ ???

Dalla conclusione precedente (hit-and-miss migliora in rapidità/semplicità rispetto a trapezi aumentando $n$) procediamo col solo hit-and-miss.

Poiché ci serve un solo metodo, inseriamo le generazione dei $n$ numeri casuali nella definizione di sfera $n$ dimensionale.

In [ ]:
def sphere_nd(n,lowlim,higlim) :
    if (lowlim > higlim):
        raise Exception('function low limit is larger than high limit')
    my_sum = sum([(random.uniform(lowlim,higlim))**2 for _ in range(n)])
    if (my_sum > 1.) : return 0.
    return np.sqrt(1-my_sum)            

## we need to know the function maximum (1 in this case)
def hit_and_miss_nd(n, lowlim, higlim, ncalls, fmax=1.) :
    hit = 0
    for i in range(ncalls) :
        r2 = random.uniform(0.,fmax)
        if (r2 < sphere_nd(n,lowlim,higlim)) :
            hit += 1 
    return (hit / ncalls) * ( higlim - lowlim )**n * fmax

In [ ]:
test4 = hit_and_miss_nd(3,-1.,1,1_000_000)

In [ ]:
print (f'{test4:.30f}')

In [ ]:
test4_in_units_pi2 = test4/(np.pi**2)

In [ ]:
print (f'{test4_in_units_pi2:.30f}')    ## true : 1/4

In [ ]:
test5 = hit_and_miss_nd(4,-1.,1,1_000_000)

In [ ]:
print (f'{test5:.30f}')

In [ ]:
test5_in_units_pi2 = test5/(np.pi**2)

In [ ]:
print (f'{test5_in_units_pi2:.30f}')     # true: 4/15 = 0.2(6)